In [26]:
# To store the data
import pandas as pd

# To do linear algebra
import numpy as np

# To create plots
import matplotlib.pyplot as plt
import seaborn as sns

# To create interactive plots
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

# To shift lists
from collections import deque

# To compute similarities between vectors
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# To use recommender systems
import surprise as sp
from surprise.model_selection import cross_validate

# To create deep learning models
from keras.layers import Input, Embedding, Reshape, Dot, Concatenate, Dense, Dropout
from keras.models import Model

# To create sparse matrices
from scipy.sparse import coo_matrix

# To light fm
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

# To stack sparse matrices
from scipy.sparse import vstack
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [3]:
#Als Nächstes sollen alle Pfade mit variablen belegt werden, dass macht das austauschen einfacher.

movie_tile_File = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/movie_titles.csv'
movie_tile_File_new = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/movie_titles_new.csv'
combined_data_1 = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/combined_data_1.txt'
combined_data_2 = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/combined_data_2.txt'
combined_data_3 = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/combined_data_3.txt'
combined_data_4 = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/combined_data_4.txt'
new_Combined = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/Combined_Data_All.gzip'
netflix_rating_Combined = 'C:/Users/jsbreite\OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/netflix_data.csv'
parquet_combined_data = 'C:/Users/jsbreite\OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/data_Comb.zip'

In [3]:
#Load Data into viables

combined_data_1_raw = pd.read_csv(combined_data_1, header=None, names=['Cust_Id', 'Rating'], usecols=[0, 1])
combined_data_2_raw = pd.read_csv(combined_data_2, header=None, names=['Cust_Id', 'Rating'], usecols=[0, 1])
combined_data_3_raw = pd.read_csv(combined_data_3, header=None, names=['Cust_Id', 'Rating'], usecols=[0, 1])
combined_data_4_raw = pd.read_csv(combined_data_4, header=None, names=['Cust_Id', 'Rating'], usecols=[0, 1])

In [4]:
## Combined_DATA_1
# Find empty rows to slice dataframe for each movie
tmp_movies = combined_data_1_raw[combined_data_1_raw['Rating'].isna()]['Cust_Id'].reset_index()  
movie_indices = [[index, int(movie[:-1])] for index, movie in tmp_movies.values]

# Shift the movie_indices by one to get start and endpoints of all movies
shifted_movie_indices = deque(movie_indices)
shifted_movie_indices.rotate(-1)


# Gather all dataframes
user_data = []

# Iterate over all movies
for [df_id_1, movie_id], [df_id_2, next_movie_id] in zip(movie_indices, shifted_movie_indices):
    
    # Check if it is the last movie in the file
    if df_id_1<df_id_2:
        tmp_df = combined_data_1_raw.loc[df_id_1+1:df_id_2-1].copy()
    else:
        tmp_df = combined_data_1_raw.loc[df_id_1+1:].copy()
        
    # Create movie_id column
    tmp_df['Movie_Id'] = movie_id
    
    # Append dataframe to list
    user_data.append(tmp_df)

# Combine all dataframes
df_1 = pd.concat(user_data)
del user_data, combined_data_1_raw, combined_data_1, tmp_movies, tmp_df, shifted_movie_indices, movie_indices, df_id_1, movie_id, df_id_2, next_movie_id
# print('Shape User-Ratings:\t{}'.format(df_1.shape))

## Combined_DATA_2
# Find empty rows to slice dataframe for each movie
tmp_movies = combined_data_2_raw[combined_data_2_raw['Rating'].isna()]['Cust_Id'].reset_index()  ##---> Nachvollziehen
movie_indices = [[index, int(movie[:-1])] for index, movie in tmp_movies.values]

# Shift the movie_indices by one to get start and endpoints of all movies
shifted_movie_indices = deque(movie_indices)
shifted_movie_indices.rotate(-1)


# Gather all dataframes
user_data = []

# Iterate over all movies
for [df_id_1, movie_id], [df_id_2, next_movie_id] in zip(movie_indices, shifted_movie_indices):
    
    # Check if it is the last movie in the file
    if df_id_1<df_id_2:
        tmp_df = combined_data_2_raw.loc[df_id_1+1:df_id_2-1].copy()
    else:
        tmp_df = combined_data_2_raw.loc[df_id_1+1:].copy()
        
    # Create movie_id column
    tmp_df['Movie_Id'] = movie_id
    
    # Append dataframe to list
    user_data.append(tmp_df)

# Combine all dataframes
df_2 = pd.concat(user_data)
del user_data, combined_data_2_raw, combined_data_2, tmp_movies, tmp_df, shifted_movie_indices, movie_indices, df_id_1, movie_id, df_id_2, next_movie_id
# print('Shape User-Ratings:\t{}'.format(df_2.shape))

## Combined_DATA_3
# Find empty rows to slice dataframe for each movie
tmp_movies = combined_data_3_raw[combined_data_3_raw['Rating'].isna()]['Cust_Id'].reset_index()  ##---> Nachvollziehen
movie_indices = [[index, int(movie[:-1])] for index, movie in tmp_movies.values]

# Shift the movie_indices by one to get start and endpoints of all movies
shifted_movie_indices = deque(movie_indices)
shifted_movie_indices.rotate(-1)


# Gather all dataframes
user_data = []

# Iterate over all movies
for [df_id_1, movie_id], [df_id_2, next_movie_id] in zip(movie_indices, shifted_movie_indices):
    
    # Check if it is the last movie in the file
    if df_id_1<df_id_2:
        tmp_df = combined_data_3_raw.loc[df_id_1+1:df_id_2-1].copy()
    else:
        tmp_df = combined_data_3_raw.loc[df_id_1+1:].copy()
        
    # Create movie_id column
    tmp_df['Movie_Id'] = movie_id
    
    # Append dataframe to list
    user_data.append(tmp_df)

# Combine all dataframes
df_3 = pd.concat(user_data)
del user_data, combined_data_3_raw, combined_data_3, tmp_movies, tmp_df, shifted_movie_indices, movie_indices, df_id_1, movie_id, df_id_2, next_movie_id
# print('Shape User-Ratings:\t{}'.format(df_3.shape))

## Combined_DATA_4
# Find empty rows to slice dataframe for each movie
tmp_movies = combined_data_4_raw[combined_data_4_raw['Rating'].isna()]['Cust_Id'].reset_index()  ##---> Nachvollziehen
movie_indices = [[index, int(movie[:-1])] for index, movie in tmp_movies.values]

# Shift the movie_indices by one to get start and endpoints of all movies
shifted_movie_indices = deque(movie_indices)
shifted_movie_indices.rotate(-1)


# Gather all dataframes
user_data = []

# Iterate over all movies
for [df_id_1, movie_id], [df_id_2, next_movie_id] in zip(movie_indices, shifted_movie_indices):
    
    # Check if it is the last movie in the file
    if df_id_1<df_id_2:
        tmp_df = combined_data_4_raw.loc[df_id_1+1:df_id_2-1].copy()
    else:
        tmp_df = combined_data_4_raw.loc[df_id_1+1:].copy()
        
    # Create movie_id column
    tmp_df['Movie_Id'] = movie_id
    
    # Append dataframe to list
    user_data.append(tmp_df)

# Combine all dataframes
df_4 = pd.concat(user_data)
del user_data, combined_data_4_raw, combined_data_4, tmp_movies, tmp_df, shifted_movie_indices, movie_indices, df_id_1, movie_id, df_id_2, next_movie_id
print('Shape User-Ratings:\t{}'.format(df_4.shape))

#Zusammenfügen der aller Daten in einer Variable
data = [df_1, df_2,df_3,df_4]
df = pd.concat(data)
del df_1, df_2, df_3, df_4



Shape User-Ratings:	(26847523, 3)


In [ ]:
movie_tile_File = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/movie_titles.csv'

In [5]:
df.to_parquet(new_Combined, index= 'False', compression= 'gzip')

In [4]:
data = pd.read_parquet(new_Combined)

In [25]:
# Load data for all movies
movie_titles = pd.read_csv(movie_tile_File, 
                           encoding = 'ISO-8859-1', 
                           engine = 'python',
                           delimiter =',',
                           header = None, 
                           on_bad_lines= 'skip', #Wird gebraucht um Fehlerhafte Title zu behen.

                           names = ['Id', 'Year', 'Name'])#.set_index('Id')
                           

print('Shape Movie-Titles:\t{}'.format(movie_titles.shape))

#Zwischenspeichern der Movies, um fehlerhafte Liste zu bearbeiten
movie_titles.to_csv(movie_tile_File_new,encoding = 'ISO-8859-1',sep=';')

movie_titles.rename(columns={'Id':'Movie_Id'}, inplace=True)
movie_titles = movie_titles.reset_index(drop=True)

display(movie_titles)

Shape Movie-Titles:	(17434, 3)


,Movie_Id,Year,Name
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW
...,...,...,...
17429,17766,2002.0,Where the Wild Things Are and Other Maurice Se...
17430,17767,2004.0,Fidel Castro: American Experience
17431,17768,2000.0,Epoch
17432,17769,2003.0,The Company


In [24]:
display(movie_titles.head(1))

,Year,Name
0,2003.0,Dinosaur Planet


In [13]:
num_users = len(data.Cust_Id.unique())
num_items = len(data.Movie_Id.unique())
print('There are {} unique users and {} unique movies in this data set'.format(num_users, num_items))

There are 480189 unique users and 17770 unique movies in this data set


In [14]:
df_ratings_cnt_tmp = pd.DataFrame(data.groupby('Rating').size(), columns=['count'])
df_ratings_cnt_tmp

,count
Rating,
1.0,4617990
2.0,10132080
3.0,28811247
4.0,33750958
5.0,23168232


In [11]:
data = data.reset_index(drop=True)
print(data.head(3))

   Cust_Id  Rating  Movie_Id
0  1488844     3.0         1
1   822109     5.0         1
2   885013     4.0         1


In [17]:
# get rating frequency
df_movies_cnt = pd.DataFrame(data.groupby('Movie_Id').size(), columns=['count'])
df_movies_cnt.head()
df_movies_cnt['count'].quantile(np.arange(1, 0.6, -0.05))

1.00    232944.00
0.95     29199.60
0.90     12303.80
0.85      6638.85
0.80      4040.20
0.75      2667.75
0.70      1948.30
0.65      1360.00
Name: count, dtype: float64

In [18]:
# filter data
popularity_thres = 10000
popular_movies = list(set(df_movies_cnt.query('count >= @popularity_thres').index))
df_ratings_drop_movies = data[data.Movie_Id.isin(popular_movies)]
print('shape of original ratings data: ', data.shape)
print('shape of ratings data after dropping unpopular movies: ', df_ratings_drop_movies.shape)

shape of original ratings data:  (100480507, 3)
shape of ratings data after dropping unpopular movies:  (79909265, 3)


In [19]:
# get number of ratings given by every user
df_users_cnt = pd.DataFrame(df_ratings_drop_movies.groupby('Cust_Id').size(), columns=['count'])
df_users_cnt.head()

,count
Cust_Id,
10,222
100000,9
1000004,28
1000027,92
1000033,371


In [20]:
df_users_cnt['count'].quantile(np.arange(1, 0.5, -0.05))

1.00    2032.0
0.95     616.0
0.90     447.0
0.85     345.0
0.80     274.0
0.75     220.0
0.70     179.0
0.65     146.0
0.60     120.0
0.55      98.0
Name: count, dtype: float64

In [22]:
# filter data
ratings_thres = 500
active_users = list(set(df_users_cnt.query('count >= @ratings_thres').index))
df_ratings_drop_users = df_ratings_drop_movies[df_ratings_drop_movies.Cust_Id.isin(active_users)]
print('shape of original ratings data: ', data.shape)
print('shape of ratings data after dropping both unpopular movies and inactive users: ', df_ratings_drop_users.shape)

shape of original ratings data:  (100480507, 3)
shape of ratings data after dropping both unpopular movies and inactive users:  (27964252, 3)


In [ ]:
movie_titles

In [28]:
# pivot and create movie-user matrix
movie_user_mat = df_ratings_drop_users.pivot(index='Movie_Id', columns='Cust_Id', values='Rating').fillna(0)

# create mapper from movie title to index
movie_to_idx = {
    movie: i for i, movie in 
    enumerate(list(movie_titles.set_index('Movie_Id').loc[movie_user_mat.index].Name))
}
# transform matrix to scipy sparse matrix
movie_user_mat_sparse = csr_matrix(movie_user_mat.values)

KeyError: '[2178, 2462, 3290, 5087, 5223, 5496, 5544, 5692, 5778, 6110, 6281, 6408, 7767, 8806, 10044, 11089, 11405, 13977, 14584, 15424, 15764, 16272, 16361, 17295] not in index'